In [ ]:
'''
# Helper to get the top models for retraining
pd.options.display.max_rows = 50
rankedTasksDict['case_inz_entries_7dayAverage'].head(8).to_csv("top_sklearn/case_inz_entries_7dayAverage.csv")
rankedTasksDict['hosp_inz_entries_7dayAverage'].head(6).to_csv("top_sklearn/hosp_inz_entries_7dayAverage.csv")
temp = rankedTasksDict['death_inz_entries_7dayAverage']
temp = temp[temp['overallRank']<=30]
temp.to_csv("top_sklearn/death_inz_entries_7dayAverage.csv")
rankedTasksDict['workplaces_percent_change_from_baseline_7dayAverage'].head(7).to_csv("top_sklearn/workplaces_percent_change_from_baseline_7dayAverage.csv")
'''

In [ ]:
'''
# model selection process (relative performance scaling and maximizing distance to origin)
from sklearn.preprocessing import MinMaxScaler

resultsCopy = results.copy()
resultsCopy = resultsCopy.drop_duplicates(['modelId'])
resultsCopy.drop(['task','Unnamed: 0','week','last value baseline 1 rmse','last value baseline 2 rmse','model rmse 1','model rmse 2'], axis=1, inplace=True)
attributesByModelId = resultsCopy.set_index('modelId')



pd.options.display.max_rows = 10

resultsCopy = results.copy()
resultsCopy = resultsCopy[['modelId','task','week','model rmse 1','model rmse 2','last value baseline 1 rmse','last value baseline 2 rmse']]

# shift week number by one
resultsCopy['week'] = resultsCopy['week']+1


tasksDict = {}


#'testPositvity_7dayAverageBoth',
for task in ['case_inz_entries_7dayAverage',
             'hosp_inz_entries_7dayAverage',
             'death_inz_entries_7dayAverage',
             'testPositvity_7dayAverage',
             'transit_stations_percent_change_from_baseline_7dayAverage',
             'workplaces_percent_change_from_baseline_7dayAverage'
                   ]:
    display(task)
    # filter by task
    taskResults = resultsCopy[resultsCopy['task']==task]
    display(taskResults.shape)
    
    # sum up the same machine learning models over all 4 weeks
    taskResults = taskResults.groupby(['modelId']).sum()
    
    # add relative score
    taskResults['rel. diff. to last value baseline 1'] = (taskResults['last value baseline 1 rmse']-taskResults['model rmse 1'])/taskResults['last value baseline 1 rmse']
    taskResults['rel. diff. to last value baseline 2'] = (taskResults['last value baseline 2 rmse']-taskResults['model rmse 2'])/taskResults['last value baseline 2 rmse']
    taskResults = taskResults.sort_values(['model rmse 1'], ascending=True)
    
    
    # add the attribute data for the models
    taskResults = taskResults.join(attributesByModelId)
    
    
    
    # filter so that only models which perform better than the baseline in both validation sets are consideres
    taskResults = taskResults[(taskResults['rel. diff. to last value baseline 1'] >= 0) & (taskResults['rel. diff. to last value baseline 2'] >= 0)]
    
    
    # only consider certain model classes
    # taskResults = taskResults[(taskResults['modelClass'] != 'Keras')]
    

    # selection algorithm
    scaler = MinMaxScaler()
    taskResults['rel. diff. to last value baseline 1 scaled'] = scaler.fit_transform(np.array(taskResults['rel. diff. to last value baseline 1']).reshape(-1, 1))
    taskResults['rel. diff. to last value baseline 2 scaled'] = scaler.fit_transform(np.array(taskResults['rel. diff. to last value baseline 2']).reshape(-1, 1))
    taskResults['distanceFromOrigin'] = (taskResults['rel. diff. to last value baseline 1 scaled']**2 + taskResults['rel. diff. to last value baseline 2 scaled']**2)**(1/2)
    taskResults.sort_values(by='distanceFromOrigin', ascending=False, inplace=True)

    
    # show the distribution of the relative performance for those models
    taskResults[["rel. diff. to last value baseline 1"]].plot.hist(by=None, bins=100, title=task+" performance distribution for validation set 1")
    taskResults[["rel. diff. to last value baseline 2"]].plot.hist(by=None, bins=100, title=task+" performance distribution for validation set 2")
    plt.show()
    #'modelIdNumber'
    display(taskResults.head(10)[['rel. diff. to last value baseline 1','rel. diff. to last value baseline 2','distanceFromOrigin','modelClass','model rmse 1','model rmse 2','modelIdNumber','numberOfRanEpochs']])
    
    # safe the summarized model results in dictionary
    tasksDict[task] = taskResults.copy()
    
    display("---------------------------------")
   
    
'''

In [ ]:
'''
lists = []
for task in ['case_inz_entries_7dayAverage',
                  'hosp_inz_entries_7dayAverage',
                  'death_inz_entries_7dayAverage',
                  'testPositvity_7dayAverageBoth',
                  'transit_stations_percent_change_from_baseline_7dayAverage',
                  'workplaces_percent_change_from_baseline_7dayAverage'
                   ]:
    display(task)
    df = tasksDict[task]
    
    
    
    # Keras
    
    #df = df[(df['isTwoWay']==False)
    #       & (df['numberOfhiddenLayers']!=8)
    #       & (df['isMultiWeek']==True)
    #       & (df['numberOfhiddenLayers']!=7)
    #       & (df['alpha']!=10)
    #       & (df['alpha']!=100)
    #       & (df['l1reg']!=10)
    #       & (df['l1reg']!=100)
    #       & (df['numberOfhiddenLayers']!=6)]
   
    
    
    
    for parameter in ['hiddelLayers']:
    #for parameter in ['isMultiWeek','numberOfhiddenLayers','hiddelLayers','isTwoWay','l1reg','alpha','dropoutValue','learningRate']:
        display("--------"+parameter+"-------")
        for error in ['distanceFromOrigin']:
        #for error in ['model rmse 1', 'model rmse 2']:
            lists.append(df[df['modelClass']=='Keras'][[error,parameter]].pivot(columns=parameter, values=error).describe().sort_values(by='max', axis=1, ascending=False).iloc[:,-40:].columns.tolist())
            display(df[df['modelClass']=='Keras'][[error,parameter]].pivot(columns=parameter, values=error).describe().sort_values(by='max', axis=1, ascending=False).iloc[:,-40:].columns.tolist())
            #display(df[df['modelClass']=='Keras'][[error,parameter]].pivot(columns=parameter, values=error).describe().sort_values(by='max', axis=1, ascending=False))
            #display(df[df['modelClass']=='Keras'][[error,parameter]].pivot(columns=parameter, values=error).describe().sort_values(by='75%', axis=1, ascending=False))
            #display(df[df['modelClass']=='Keras'][[error,parameter]].pivot(columns=parameter, values=error).describe().sort_values(by='50%', axis=1, ascending=False))
            #df[df['modelClass']=='Keras'][[error,parameter]].pivot(columns=parameter, values=error).plot(kind="box", title = parameter+" effect "+error, ylabel=error, xlabel=parameter, figsize=(10,5))
            #plt.show()       
    
    
    display("---------------------------------------------")

def intersection(lst1, lst2):
    return list(set(lst1) & set(lst2))

#display(intersection(lists[0],lists[1]))
init = intersection(lists[0],lists[1])
for i in range(1,len(lists)-1):
    init = intersection(init, lists[i])
display(init)
'''

In [ ]:
'''
# inspecting models by hyperparamters
for task in ['case_inz_entries_7dayAverage',
                  'hosp_inz_entries_7dayAverage',
                  'death_inz_entries_7dayAverage',
                  'testPositvity_7dayAverage',
                  'transit_stations_percent_change_from_baseline_7dayAverage',
                  'workplaces_percent_change_from_baseline_7dayAverage'
                   ]:
    display(task)
    df = tasksDict[task]
    df = df[df['modelClass']=='Keras']
    parameterOfInterest = 'numberOfhiddenLayers' #'isMultiWeek','numberOfhiddenLayers','hiddelLayers','isTwoWay','l1reg','alpha','dropoutValue','learningRate'
    df = df[['model rmse 1','model rmse 2', parameterOfInterest]]
    
    plt.figure(figsize=(15,15))
    for possibleValue in df[parameterOfInterest].unique():
        plt.scatter(df['model rmse 1'][df[parameterOfInterest] == possibleValue],
                df['model rmse 2'][df[parameterOfInterest] == possibleValue],
               marker='o',
               label=possibleValue,
               alpha=0.4)
    plt.xlabel('model rmse 1')
    plt.ylabel('model rmse 2')
    plt.legend()

    plt.show()
   
    display("---------------------------------------------")
'''  

In [ ]:
'''
# all models plot
for task in ['case_inz_entries_7dayAverage',
                  'hosp_inz_entries_7dayAverage',
                  'death_inz_entries_7dayAverage',
                  'testPositvity_7dayAverage',
                  'transit_stations_percent_change_from_baseline_7dayAverage',
                  'workplaces_percent_change_from_baseline_7dayAverage'
                   ]:
    display(task)
    df = tasksDict[task]
    df = df[['model rmse 1','model rmse 2','modelClass']]
    display(df.shape)
    
    df = df[(df['model rmse 1'] < df['model rmse 1'].mean()) & 
           (df['model rmse 2'] < df['model rmse 2'].mean())
           ]
    df = df[(df['model rmse 1'] < df['model rmse 1'].mean() + 3*df['model rmse 1'].std()) & 
           (df['model rmse 2'] < df['model rmse 2'].mean() + 3*df['model rmse 2'].std())
           ]
    display(df.shape)
    #display(df['model rmse 1'].mean())
    #+ 3*df['model rmse 1'].std()

    
    plt.figure(figsize=(15,15))
    plt.scatter(df['model rmse 1'][df.modelClass == 'Keras'],
            df['model rmse 2'][df.modelClass == 'Keras'],
           marker='o',
           color='blue',
           label='Keras',
           alpha=0.2)
    plt.scatter(df['model rmse 1'][df.modelClass == 'LGBM'],
            df['model rmse 2'][df.modelClass == 'LGBM'],
           marker='o',
           color='red',
           label='LGBM',
           alpha=0.4)
    plt.scatter(df['model rmse 1'][df.modelClass == 'XGB'],
            df['model rmse 2'][df.modelClass == 'XGB'],
           marker='o',
           color='green',
           label='XGB',
           alpha=0.4)
    plt.scatter(df['model rmse 1'][df.modelClass == 'RandomForrest'],
            df['model rmse 2'][df.modelClass == 'RandomForrest'],
           marker='o',
           color='yellow',
           label='RandomForrest',
           alpha=0.4)
    
    plt.scatter(df['model rmse 1'][df.modelClass == 'Ridge'],
            df['model rmse 2'][df.modelClass == 'Ridge'],
           marker='o',
           label='Ridge',
           alpha=0.4)
    
    plt.scatter(df['model rmse 1'][df.modelClass == 'Lasso'],
            df['model rmse 2'][df.modelClass == 'Lasso'],
           marker='o',
           label='Lasso',
           alpha=0.4)
    
    plt.scatter(df['model rmse 1'][df.modelClass == 'MultiTaskLasso'],
            df['model rmse 2'][df.modelClass == 'MultiTaskLasso'],
           marker='o',
           label='MultiTaskLasso',
           alpha=0.4)
    
    plt.scatter(df['model rmse 1'][df.modelClass == 'SVR'],
            df['model rmse 2'][df.modelClass == 'SVR'],
           marker='o',
           label='SVR',
           alpha=0.4)
    
    
    plt.scatter(df['model rmse 1'][df.modelClass == 'SGD'],
            df['model rmse 2'][df.modelClass == 'SGD'],
           marker='o',
           label='SGD',
           alpha=0.4)
    ''
    plt.scatter(df['model rmse 1'][df.modelClass == 'KernelRidge'],
            df['model rmse 2'][df.modelClass == 'KernelRidge'],
           marker='o',
           label='KernelRidge',
           alpha=0.4)
   
    plt.xlabel('model rmse 1')
    plt.ylabel('model rmse 2')
    plt.legend()

    plt.show()
   
    display("---------------------------------------------")
'''

In [ ]:
'''
# SINGLE VS. MULTI OUTPUT (for all weeks and for all targets plot single, multi, baseline per model class)
for modelClass in ['RandomForrest']: # 'RandomForrest', 'Lasso''Keras' ,'SGD','KernelRidge','MultiTaskLasso','MLP','KernelRidge','XGB','LGBM','Ridge','SVR','Keras'
    for category in outputCategories:
        merged1 = pd.DataFrame()
        merged2 = pd.DataFrame()
        for week in range(0,numberOfOutputWeeks):
            temp = df[(df['week']==week)&(df['task']==category)&(df['modelClass']==modelClass)].groupby(['isMultiWeek']).apply(lambda x: x.sort_values(["rel. diff. to last value baseline 1"], ascending=False)).reset_index(drop=True)
            temp = temp.groupby(['isMultiWeek']).head(1)
            temp1 = temp[['model rmse 1', 'last value baseline 1 rmse', 'isMultiWeek']].set_index('isMultiWeek')
            temp2 = temp[['model rmse 2', 'last value baseline 2 rmse', 'isMultiWeek']].set_index('isMultiWeek')
            new_row1 = pd.Series(data={'rmse':temp1['last value baseline 1 rmse'].values[0]}, name='baseline')
            new_row2 = pd.Series(data={'rmse':temp2['last value baseline 2 rmse'].values[0]}, name='baseline')
            temp1.rename(columns = {'model rmse 1':'rmse'}, inplace = True)
            temp2.rename(columns = {'model rmse 2':'rmse'}, inplace = True)
            temp1.drop(['last value baseline 1 rmse'], inplace=True, axis=1)
            temp2.drop(['last value baseline 2 rmse'], inplace=True, axis=1)
            temp1 = temp1.append(new_row1, ignore_index=False) #.sort_values(['rsme'], ascending=False)
            temp2 = temp2.append(new_row2, ignore_index=False)
            temp1.rename(columns = {'rmse':str(week+1)}, inplace=True)
            temp2.rename(columns = {'rmse':str(week+1)}, inplace=True)
            merged1 = merged1.append(temp1.transpose())
            merged2 = merged2.append(temp2.transpose())
            #temp.plot(kind='bar', title=modelClass+"_"+category+"_"+"week_"+str(week))
            #plt.show()
        merged1.plot(kind='line',title=modelClass+" "+category +" (validation set 1 scores)",ylabel='rmse', xlabel="week", figsize=(20,10))
        merged2.plot(kind='line',title=modelClass+" "+category +" (validation set 2 scores)",ylabel='rmse', xlabel="week", figsize=(20,10))
'''

In [ ]:
'''
# MODEL COMPARISON (for all weeks and for all target best estimator per model class)
pd.options.display.max_rows = 11
for category in outputCategories:
    mergedAbs1 = pd.DataFrame()
    mergedAbs2 = pd.DataFrame()
    mergedRel1 = pd.DataFrame()
    mergedRel2 = pd.DataFrame()
    for week in range(0,numberOfOutputWeeks):
        temp = df[(df['week']==week)&(df['task']==category)].groupby(['modelClass']).apply(lambda x: x.sort_values(["rel. diff. to last value baseline 1"], ascending=False)).reset_index(drop=True)
        temp = temp.groupby(['modelClass']).head(1) #.sort_values(['rel. diff. to last value baseline 1'], ascending=False)
        
        # for absolut values plot
        temp1V1 = temp[['modelClass','model rmse 1','last value baseline 1 rmse']].set_index('modelClass')
        temp1V2 = temp[['modelClass','model rmse 2','last value baseline 2 rmse']].set_index('modelClass') 
        new_row1 = pd.Series(data={'rmse':temp1V1['last value baseline 1 rmse'].values[0]}, name='baseline')
        new_row2 = pd.Series(data={'rmse':temp1V2['last value baseline 2 rmse'].values[0]}, name='baseline')
        temp1V1.rename(columns = {'model rmse 1':'rmse'}, inplace = True)
        temp1V2.rename(columns = {'model rmse 2':'rmse'}, inplace = True)
        temp1V1.drop(['last value baseline 1 rmse'], inplace=True, axis=1)
        temp1V2.drop(['last value baseline 2 rmse'], inplace=True, axis=1)
        temp1V1 = temp1V1.append(new_row1, ignore_index=False).sort_values(['rmse'], ascending=False)
        temp1V2 = temp1V2.append(new_row2, ignore_index=False).sort_values(['rmse'], ascending=False)
        temp1V1.rename(columns = {'rmse':str(week+1)}, inplace=True)
        temp1V2.rename(columns = {'rmse':str(week+1)}, inplace=True)
        mergedAbs1 = mergedAbs1.append(temp1V1.transpose())
        mergedAbs2 = mergedAbs2.append(temp1V2.transpose())

        # for relative to baseline plot
        temp2V1 = temp[['modelClass','rel. diff. to last value baseline 1']].set_index('modelClass')
        temp2V2 = temp[['modelClass','rel. diff. to last value baseline 2']].set_index('modelClass')
        temp2V1.rename(columns = {'rel. diff. to last value baseline 1':'rel. diff. to baseline'}, inplace = True)
        temp2V2.rename(columns = {'rel. diff. to last value baseline 2':'rel. diff. to baseline'}, inplace = True)
        temp2V1 = temp2V1.sort_values(['rel. diff. to baseline'], ascending=True)
        temp2V2 = temp2V2.sort_values(['rel. diff. to baseline'], ascending=True)
        temp2V1.rename(columns = {'rel. diff. to baseline':str(week+1)}, inplace=True)
        temp2V2.rename(columns = {'rel. diff. to baseline':str(week+1)}, inplace=True)
        mergedRel1 = mergedRel1.append(temp2V1.transpose())
        mergedRel2 = mergedRel2.append(temp2V2.transpose())

    mergedAbs1.plot(kind='line',title=category + " (validation set 1 scores)",ylabel='rmse', xlabel="week", figsize=(20,10))
    mergedAbs2.plot(kind='line',title=category + " (validation set 2 scores)",ylabel='rmse', xlabel="week", figsize=(20,10))
    mergedRel1.plot(kind='line',title=category + " (validation set 1 scores)",ylabel='rel. diff. to baseline', xlabel="week", figsize=(20,10))
    mergedRel2.plot(kind='line',title=category + " (validation set 2 scores)",ylabel='rel. diff. to baseline', xlabel="week", figsize=(20,10))
    plt.show()
'''

In [ ]:
'''
# OVERVIEW (for all weeks best estimator per target)
df = results.copy()
merged1 = pd.DataFrame()
merged2 = pd.DataFrame()
for week in range(0,numberOfOutputWeeks):
    temp = df[df['week']==week].groupby(['task']).apply(lambda x: x.sort_values(['rel. diff. to last value baseline 1'], ascending=False)).reset_index(drop=True)
    temp = temp.groupby(['task']).head(1)
    temp1 = temp[['task','rel. diff. to last value baseline 1']].set_index('task')
    temp2 = temp[['task','rel. diff. to last value baseline 2']].set_index('task')
    temp1.rename(columns = {'rel. diff. to last value baseline 1': str(week+1)}, inplace = True)
    temp2.rename(columns = {'rel. diff. to last value baseline 2': str(week+1)}, inplace = True)
    merged1 = merged1.append(temp1.transpose())
    merged2 = merged2.append(temp2.transpose())
    #.sort_values(['rsme'], ascending=False)
    #temp.plot(kind='bar', title="week_"+str(week))
    #plt.show()
    
merged1.plot(kind='line', title="validation set 1 scores", ylabel='rel. diff. to baseline', xlabel="week", figsize=(10,5))
merged2.plot(kind='line', title="validation set 2 scores", ylabel='rel. diff. to baseline', xlabel="week", figsize=(10,5))
'''

In [ ]:
'''
file1 = open('output.log', 'r')
Lines = file1.readlines()
 


estimatorErrors = []

for line in Lines:
    estimatorErrors.append(int(line[32:-11]))

   

with open('erstimatorErrors.txt', 'w') as f:
    for e in estimatorErrors:
        f.write(str(e)+"\n")
display(len(estimatorErrors))


for e in estimatorErrors[0:125]:
    display(str(e)+": "+str(estimators[e]))
  
'''
